# AMOC modelling 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import normal, randn

## Part 1 : Consider F(t)=F=constant and study the multistability of the model
### 1.1 : Take F=3.24 m year-1. Show graphically that the system is bistable.
This is equivalent to solving $\frac{d\Delta S}{dt} = 0$ and see when the line crosses $x=0$


In [ ]:
### define all constants ###
H  = 4500           # [m]
S0 = 35             # [psu]
tho_d = 219         # [years]
L = 8250e3
sigmaw = 300e3
V = L*H*sigmaw      # [m^3]
q = 3.27e19         # [m^3 years^-1]
alpha_s = 0.75e-3   # [psu^-1]
alpha_t = 0.17e-3   # [°C^-1]
theta = 20          # [°C]

### code for 1.1 ###
F = 3.24  # [m year^-1]

# array of values of Delta S to to compute d Delta S / dt
DeltaS_val = np.arange(0, 6, 0.1)

# compute using the right-hand side of the equation
DS = (F*S0/H) - (DeltaS_val/tho_d) - (q*DeltaS_val*(alpha_s*DeltaS_val - alpha_t*theta)**2)/V


In [ ]:
plt.figure()
plt.plot(DeltaS_val,DS)
plt.xlabel('$\Delta S$')
plt.ylabel('$d \Delta S/dt$')
plt.axhline(0, c='k', ls='--')
plt.show()


In [ ]:
V = - np.cumsum(DS)*0.1
plt.figure()
plt.ylabel('V')
plt.xlabel('$ \Delta S$')
plt.plot(DeltaS_val, V)
plt.show()

### 1.2 : Cessi proposes F=2.3 m year-1. Is the system bistable in this case? Estimate numerically the critical value of F at which the system changes the number of stable solutions

In [ ]:
Fvals = np.arange(2.3, 3.5, 0.1)
all_DS = np.zeros((len(DeltaS_val), len(Fvals)))
print(all_DS)
for i in range(len(Fvals)):
    all_DS[:,i] = (Fvals[i]*S0/H) - (DeltaS_val/tho_d) - (q*DeltaS_val*(alpha_s*DeltaS_val - alpha_t*theta)**2)/V
print(all_DS)

f = plt.figure()
plt.plot(DeltaS_val, all_DS)
plt.axhline(0, c='k', ls='--')
plt.xlabel('$\Delta S$')
plt.ylabel('$d \Delta S/dt$')
f.legend([str(round(F,2)) for F in Fvals])
plt.show()

In [ ]:
flag = True
F = 2.3

while (flag):
    DeltaS_values = np.arange(0, 6, 0.001)
    DS = (F/H)*S0 - (DeltaS_values/tho_d) - (q*DeltaS_values*(alpha_s*DeltaS_values - alpha_t*theta)**2)/V
    Number = (np.diff(np.sign(DS)) != 0).sum()
    
    if(Number > 1):
        flag = False
    F += 0.0001
print("for F = {}, we cross 0 : {} times".format(round(F,10), Number))


threshold_DS = (F*S0/H) - (DeltaS_val/tho_d) - (q*DeltaS_val*(alpha_s*DeltaS_val - alpha_t*theta)**2)/V

plt.figure()
plt.plot(DeltaS_val, threshold_DS)
plt.axhline(0, c='k', ls='--')
plt.xlabel('$\Delta S$')
plt.ylabel('$d \Delta S/dt$')
plt.show()

### 1.3 Write a code to integrate in time the equation and show plots of time series of the solutions to support what you found in points 1 and 2. You can take a timestep of 1 year.

In [ ]:
# Forward-Euler for d(DeltaS)/dt
T   = 1000
dt  = 1
DS0 = 0.1
init_vals = np.arange(0, 6, 0.5)

Flow = 2.3
Fup  = 3.4
Fthr = 2.5649

# -- Try with one stable solution : Flow
int_DS = np.zeros((T, len(init_vals)))
int_DS[0,:] = init_vals

for n in range(len(init_vals)):
    for i in range(1,T):
        rhs = (Flow*S0/H) - (int_DS[i-1,n]/tho_d) - (q*int_DS[i-1,n]*(alpha_s*int_DS[i-1,n] - alpha_t*theta)**2)/V
        int_DS[i,n] = int_DS[i-1,n] + rhs*dt

f = plt.figure()
plt.plot(np.arange(0,T,1) ,int_DS)
plt.xlabel('Time [years]')
plt.ylabel('$\Delta S$')
f.legend([str(round(init,2)) for init in init_vals])
plt.show()

# -- Try with various stable solution : Fup
int_DS = np.zeros((T, len(init_vals)))
int_DS[0,:] = init_vals

for n in range(len(init_vals)):
    for i in range(1,T):
        rhs = (Fup*S0/H) - (int_DS[i-1,n]/tho_d) - (q*int_DS[i-1,n]*(alpha_s*int_DS[i-1,n] - alpha_t*theta)**2)/V
        int_DS[i,n] = int_DS[i-1,n] + rhs*dt

f = plt.figure()
plt.plot(np.arange(0,T,1) ,int_DS)
plt.xlabel('Time [years]')
plt.ylabel('$\Delta S$')
f.legend([str(round(init,2)) for init in init_vals])
plt.show()

# -- Try with critical case : Fthreshold
int_DS = np.zeros((T, len(init_vals)))
int_DS[0,:] = init_vals

for n in range(len(init_vals)):
    for i in range(1,T):
        rhs = (Fthr*S0/H) - (int_DS[i-1,n]/tho_d) - (q*int_DS[i-1,n]*(alpha_s*int_DS[i-1,n] - alpha_t*theta)**2)/V
        int_DS[i,n] = int_DS[i-1,n] + rhs*dt

f = plt.figure()
plt.plot(np.arange(0,T,1) ,int_DS)
plt.xlabel('Time [years]')
plt.ylabel('$\Delta S$')
f.legend([str(round(init,2)) for init in init_vals])
plt.show()

## Part 2 : Consider F(t) as a stochastic forcing and study the multistability

### 2.1 Take F=3.24 m year-1 and plot the potential of the model